In [ ]:
%pylab inline
import os, glob, pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
pylab.rcParams['figure.figsize'] = (10, 6)

In [ ]:
import sys
sys.path.append("../")
import muscle_utils
import muscle

In [ ]:
import parest
import amre1, amre2
import amrf1, amrf2

In [ ]:
ts = 0.01
padding = int(10/ts)

In [ ]:
angles_in = [amre1.angles_in,
             [0]*padding,
             amre2.angles_in,
             [0]*padding,
             amrf1.angles_in,
             [0]*padding,
             amrf2.angles_in,
]


torque_out = [amre1.torque_out,
              [0]*padding,
              amre2.torque_out,
              [0]*padding,
              amrf1.torque_out,
              [0]*padding,
              amrf2.torque_out,
]


In [ ]:
emg0_training = [amre1.emg0_training,
                 np.zeros((padding, 8)),
                 amre2.emg0_training,
                 np.zeros((padding, 8)),
                 amrf1.emg0_training,
                 np.zeros((padding, 8)),
                 amrf2.emg0_training,
]
emg1_training = [amre1.emg1_training,
                 np.zeros((padding, 8)),
                 amre2.emg1_training,
                 np.zeros((padding, 8)),
                 amrf1.emg1_training,
                 np.zeros((padding, 8)),
                 amrf2.emg1_training,
]

emg0_in = [amre1.emg0_in,
           np.zeros((padding, 8)),
           amre2.emg0_in,
           np.zeros((padding, 8)),
           amrf1.emg0_in, 
           np.zeros((padding, 8)),
           amrf2.emg0_in,
]
emg1_in = [amre1.emg1_in,
           np.zeros((padding, 8)),
           amre2.emg1_in,
           np.zeros((padding, 8)),
           amrf1.emg1_in,
           np.zeros((padding, 8)),
           amrf2.emg1_in,
]


In [ ]:
imu0_in = [amre1.imu0_meas,
           np.zeros((padding, 7)),
           amre2.imu0_meas,
           np.zeros((padding, 7)),
           amrf1.imu0_meas, 
           np.zeros((padding, 7)),
           amrf2.imu0_meas,
]

imu1_in = [amre1.imu1_meas,
           np.zeros((padding, 7)),
           amre2.imu1_meas,
           np.zeros((padding, 7)),
           amrf1.imu1_meas,
           np.zeros((padding, 7)),
           amrf2.imu1_meas,
]

In [ ]:

with open('../../2muslces_cleaned.pickle', 'rb') as tmp:
    pars = pickle.load(tmp)

triceps_pars = [
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['ACTIVATION_SIGNAL']['C1'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['ACTIVATION_SIGNAL']['C2'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['ACTIVATION_SIGNAL']['A'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['ACTIVATION_SIGNAL']['d'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['max_length'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['optimal_fiber_length'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['tensor_slack_length'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['max_force'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['alpha'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['Spe'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['Sse'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['phi_m'],
    pars[muscle_utils.MUSCLE_NAME.TRICEPS_BRACHII]['phi_v'],
]

biceps_pars = [
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['ACTIVATION_SIGNAL']['C1'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['ACTIVATION_SIGNAL']['C2'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['ACTIVATION_SIGNAL']['A'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['ACTIVATION_SIGNAL']['d'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['max_length'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['optimal_fiber_length'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['tensor_slack_length'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['max_force'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['alpha'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['Spe'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['Sse'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['phi_m'],
    pars[muscle_utils.MUSCLE_NAME.BICEPS_BRACHII]['phi_v'],
]

params = triceps_pars + biceps_pars

In [ ]:
parest.angles_in = angles_in
parest.torque_out = torque_out
parest.emg0_training = emg0_training
parest.emg1_training = emg1_training
parest.emg0_in = emg0_in
parest.emg1_in = emg1_in
parest.imu0_in = imu0_in
parest.imu1_in = imu1_in

In [ ]:
torque = []
for tau_tmp in parest.torque_out:
    torque = torque + list(tau_tmp)

In [ ]:
estimated = parest.FlexProblem()
muscles = estimated.set_up_muscles(params)
tau, act, m_tau = estimated.simulate(muscles)
act = np.array(act[0:2])

In [ ]:
plt.plot(tau)
plt.plot(torque)

In [ ]:
# a little filter
nyq = 0.5 * 100
cut = 1
low = cut / nyq
b, a = signal.butter(4, low, 'low')

In [ ]:
start = len(amre1.emg0_in) + padding + 500
stop = start + len(amre2.emg0_in) - 500
ext_est = tau[start:stop]
ext = torque[start:stop]
ext_act = act[:,start:stop]
ext_time = np.linspace(0, (stop-start)*0.01, stop-start)
plt.plot(ext_time, ext)
plt.plot(ext_time, ext_est)
plt.plot(ext_time, signal.lfilter(b, a, ext_est))

In [ ]:
start = len(amre1.emg0_in) + padding +  len(amre2.emg0_in) + padding
stop = start + len(amrf1.emg0_in)
flex_est = tau[start:stop]
flex = torque[start:stop]
flex_act = act[:,start:stop]
flex_time = np.linspace(0, (stop-start)*0.01, stop-start)
plt.plot(flex_time, flex)
plt.plot(flex_time, flex_est)
plt.plot(flex_time, signal.lfilter(b, a, flex_est))

In [ ]:
HEADER = 'TIME, MEAS, EST, EST_LP, TRICEPS, BICEPS, BRACHIALIS, BRACHIORADIALIS'

In [ ]:
np.savetxt(
    'ext_2m.dat', 
    np.vstack(
        [
            ext_time, 
            np.array(ext),
            np.array(ext_est), 
            signal.lfilter(b, a, ext_est),
            ext_act
        ]
    ).transpose(), 
    header=HEADER, 
    comments=''
)

In [ ]:
np.savetxt(
    'flex_2m.dat', 
    np.vstack(
        [
            flex_time, 
            np.array(flex),
            np.array(flex_est), 
            signal.lfilter(b, a, flex_est),
            flex_act
        ]
    ).transpose(), 
    header=HEADER, 
    comments=''
)